In [55]:
import numpy as np

path = 'data_15.npy'
path2 = 'data_chb15_10.npy'
path3 = 'data_chb15_15.npy'
path4 = 'data_chb15_17.npy'

data = np.load(path)
data2 = np.load(path2)
data3 = np.load(path3)
data4 = np.load(path4)

data_add = np.append(data[0],data2[0])
data_add = np.append(data_add,data3[0])
data_add = np.append(data_add,data4[0])

label = np.append(data[2],data2[2])
label = np.append(label,data3[2])
label = np.append(label,data4[2])

print(data_add.shape, label.shape)

(3686400,) (3686400,)


In [60]:
from itertools import count


def strided_app(a, L, S ):  # Window len = L, Stride len/stepsize = S
    nrows = ((a.size-L)//S)+1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows,L), strides=(S*n,n))

x = strided_app(data_add, L = 256, S = 128)
# print(x.shape)

z = strided_app(label, L = 256, S = 128)
y = np.zeros(x.shape[0])
for i in range(0,x.shape[0]):
    if np.mean(z[i]) < 0.5:
        y[i] = 0
    elif np.mean(z[i]) <= 1:
        y[i] = 1
    else:
        y[i] = 2
non = np.count_nonzero(y==0)
soz = np.count_nonzero(y==1)
ict = np.count_nonzero(y==2)
# print(y.shape)
print('set of label "0": {}'.format(non))
print('set of label "1": {}'.format(soz))
print('set of label "2": {}'.format(ict))

set of label "0": 27859
set of label "1": 243
set of label "2": 697


In [57]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Undersampling
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import EditedNearestNeighbours

# enn = EditedNearestNeighbours()
# rus = RandomUnderSampler(random_state=0)
nr = NearMiss(n_neighbors=3)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)
X_train_resample, y_train_resample = nr.fit_resample(X_train, y_train)
X_test_resample, y_test_resample = nr.fit_resample(X_test, y_test)

print('After Undersampling, the shape of train_X: {}'.format(X_train_resample.shape))
print('After Undersampling, the shape of train_y: {} \n'.format(y_train_resample.shape))
print("After Undersampling, counts of label '0': {}".format(sum(y_train_resample == 0)))
print("After Undersampling, counts of label '1': {}".format(sum(y_train_resample == 1)))
print("After Undersampling, counts of label '2': {}\n".format(sum(y_train_resample == 2)))

print('After Undersampling, the shape of test_X: {}'.format(X_test_resample.shape))
print('After Undersampling, the shape of test_y: {}'.format(y_test_resample.shape))
print("After Undersampling, counts of label '0': {}".format(sum(y_test_resample == 0)))
print("After Undersampling, counts of label '1': {}".format(sum(y_test_resample == 1)))
print("After Undersampling, counts of label '2': {}".format(sum(y_test_resample == 2)))

# print('After Undersampling, the shape of train_X: {}'.format(X_train.shape))
# print('After Undersampling, the shape of train_y: {} \n'.format(y_train.shape))
# print("After Undersampling, counts of label '0': {}".format(sum(y_train == 0)))
# print("After Undersampling, counts of label '1': {}".format(sum(y_train == 1)))
# print("After Undersampling, counts of label '2': {}".format(sum(y_train == 2)))

After Undersampling, the shape of train_X: (516, 256)
After Undersampling, the shape of train_y: (516,) 

After Undersampling, counts of label '0': 172
After Undersampling, counts of label '1': 172
After Undersampling, counts of label '2': 172

After Undersampling, the shape of test_X: (213, 256)
After Undersampling, the shape of test_y: (213,)
After Undersampling, counts of label '0': 71
After Undersampling, counts of label '1': 71
After Undersampling, counts of label '2': 71


In [58]:
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train_resample, y_train_resample)
svm_predictions = svm_model_linear.predict(X_test_resample)

# model accuracy for X_test  
accuracy = svm_model_linear.score(X_test_resample, y_test_resample)
  
# creating a confusion matrix
cm = confusion_matrix(y_test_resample, svm_predictions)

print(accuracy)
print(cm)

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test_resample, svm_predictions, target_names=['non-seizure', 'soz', 'ictal']))


0.6244131455399061
[[71  0  0]
 [17  5 49]
 [14  0 57]]

Classification Report

              precision    recall  f1-score   support

 non-seizure       0.70      1.00      0.82        71
         soz       1.00      0.07      0.13        71
       ictal       0.54      0.80      0.64        71

    accuracy                           0.62       213
   macro avg       0.74      0.62      0.53       213
weighted avg       0.74      0.62      0.53       213

